In [1]:
# import dependencies 
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup as soup
import pandas as pd
import datetime as dt
from webdriver_manager.chrome import ChromeDriverManager
import time
import numpy as np
import os
import random

In [2]:
# open browser
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 104.0.5112
Get LATEST chromedriver version for 104.0.5112 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/104.0.5112.79/chromedriver_mac64.zip
Driver has been saved in cache [/Users/evanbruno/.wdm/drivers/chromedriver/mac64/104.0.5112.79]


In [16]:
# visit nba page 
url = 'https://www.nba.com/stats/teams/boxscores/'
browser.visit(url)

In [4]:
# url used for entering each games data
revise_url = "https://www.nba.com/game/"

In [23]:
# number of pages to click through
for pages in range(1, 10):
    
    #sleep
    time.sleep(random.randint(1, 5))
    
    html = browser.html
    test = soup(html, 'html.parser')
    
    #finds all the hyperlinks on page 
    find = test.find_all("tbody")[0].find_all("a")

    links = []
    # iterates through hyperlinks to find ones linked to game scores
    for link in find:
        if "/stats/game/" in link.get('href'):
            links.append(link.get('href')[12:])
    
    # get dates to name each csv file 
    test_date = []
    names = []
    for date in find:
        if "/games?date" in date.get('href'):
            test_date.append(date.get_text())
            names.append(f"{date.get_text()[:2]}-{date.get_text()[3:5]}")
            
    #iterate through each game link 
    for y in range(len(links)):
        #used to make sure don't click the same game link twice
        if (y % 2) == 0:
            print(y)
            #visit game link 
            browser.visit(f"{revise_url}{links[y]}")
            time.sleep(3)
            #finds box score 
            browser.find_by_id("box-score").click()
            time.sleep(random.randint(1, 5))
            html = browser.html
            test2 = soup(html, 'html.parser')
            
            #obtain column names
            column = []
            for i in range(1, len(test2.find(class_="StatsTableHead_thead__2tFet").find_all("th"))):
                column.append(test2.find(class_="StatsTableHead_thead__2tFet").find_all("th")[i].get_text())
            
            #iterate through both teams box scores
            for i in range(2):
                stats = {}
                #get team name
                team_name = test2.find_all(class_="p-4 md:p-5")[i].find(class_="block").get_text()
                
                #iterate through rows
                for t in range(len(test2.find_all("tbody", class_="StatsTableBody_tbody__2eDxB")[i].find_all("tr"))):
        
                    #iterate through columns to get data 
                    for x in range(1, 21):
                    
                        # comparison used to avoid hitting errors in obtaining wrong data 
                        if test2.find_all("tbody", class_="StatsTableBody_tbody__2eDxB")[i].find_all("tr")[t].find_all("td")[0].get_text() == "TOTALS":
                            break
                        
                        if x == 1:
                            #adds name of player to dictionary
                            name = (test2.find_all("tbody", class_="StatsTableBody_tbody__2eDxB")[i].find_all("tr")[t].find("td").find(class_="hidden md:inline-block").get_text())
                            stats[name] = []
                        
                        #append nan if player does not play  
                        if test2.find_all("tbody", class_="StatsTableBody_tbody__2eDxB")[i].find_all("tr")[t].find_all("td")[1].get_text() in ("DNP - Coach's Decision", "DNP - Injury/Illness", "NWT - Personal", "DND - Injury/Illness", "DNP - League Suspension", "DNP - Personal", "DND - Rest", "DND_LEAGUE_SUSPENSION"):
                            stats[name].append(np.nan)
                        
                        #append stats for each player 
                        else:
                            stats[name].append(test2.find_all("tbody", class_="StatsTableBody_tbody__2eDxB")[i].find_all("tr")[t].find_all("td")[x].get_text())
                # creates dataframe 
                team_data = pd.DataFrame.from_dict(stats, orient='index', columns = column)
                #creates data column
                team_data["Date"] = test_date[y]
                team_data = team_data.dropna()
                
                #creates float datatype for each column
                for x in ['FGM', 'FGA', 'FG%', '3PM', '3PA', '3P%', 'FTM', 'FTA', 'FT%',
       'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TO', 'PF', 'PTS', '+/-']:
                    team_data[x] = team_data[x].astype(float)
                    
                # calculates players score according to draftkings
                team_data["Score"] = ((team_data["FGM"] - team_data["3PM"]) * 2) * 1 + (team_data["3PM"] * 0.5) + team_data["3PM"] * 3 + team_data["REB"] * 1.25 + team_data["AST"] * 1.5 + team_data["STL"] * 2 + team_data["BLK"] * 2 - team_data["TO"] * 0.5 + team_data["FTM"]
                
                #creates folder for team
                mypath = team_name
                if not os.path.isdir(mypath):
                    os.makedirs(mypath)
                # converts dataframe to csv
                team_data.to_csv(f"{mypath}/{names[y]}.csv", index=True)
    #revisit url
    browser.visit(url)
    #click to next page
    for clicker in range(pages):
        browser.find_by_css(".stats-table-pagination__next").click()

0
2
4
6
8
10
12
14
16
18
20
22
24
26
28
30
32
34
36
38
40
42
44
46
48
0
2
4
6
8
10
12
14
16
18
20
22
24
26
28
30
32
34
36
38
40
42
44
46
48
0
2
4
6
8
10
12
14
16
18
20
22
0
2
4
6
8
10
12
14
16
18
20
22
0
2
4
6
8
10


WebDriverException: Message: disconnected: Unable to receive message from renderer
  (Session info: chrome=104.0.5112.101)
Stacktrace:
0   chromedriver                        0x000000010b922149 chromedriver + 4469065
1   chromedriver                        0x000000010b8ac233 chromedriver + 3985971
2   chromedriver                        0x000000010b542fe8 chromedriver + 409576
3   chromedriver                        0x000000010b530242 chromedriver + 332354
4   chromedriver                        0x000000010b52fa53 chromedriver + 330323
5   chromedriver                        0x000000010b54a22a chromedriver + 438826
6   chromedriver                        0x000000010b5a9a1d chromedriver + 829981
7   chromedriver                        0x000000010b596b53 chromedriver + 752467
8   chromedriver                        0x000000010b56c905 chromedriver + 579845
9   chromedriver                        0x000000010b56d955 chromedriver + 584021
10  chromedriver                        0x000000010b8

In [ ]:
[i].find_all("tr")[t].find_all("td")[x]

In [22]:
test2.find_all("tbody", class_="StatsTableBody_tbody__2eDxB")[i].find_all("tr")[t]

<tr><td class="py-1"><div class="flex items-center"><a class="flex items-center text-cerulean font-condensed Anchor_complexLink__2NtkO" href="/player/1630267/facundo-campazzo"><figure class="GameBoxscoreTablePlayer_playerImage__2vltF hidden md:block"><img alt=" Headshot" class="PlayerImage_image__1smob PlayerImage_round__281uY" loading="lazy" src="https://cdn.nba.com/headshots/nba/latest/260x190/1630267.png"/></figure><span class="md:ml-2"><span class="hidden md:inline-block">Facundo Campazzo</span><span class="md:hidden">F. Campazzo</span></span></a><span class="ml-1 md:ml-2 text-asphalt"></span></div></td><td colspan="20"><span class="block pl-2 text-left text-asphalt">DND_LEAGUE_SUSPENSION</span></td></tr>

In [9]:
html = browser.html
test2 = soup(html, 'html.parser')

In [12]:
test2.find_all(id="box-score")

[<a class="InnerNavTabLink_link__3lva8" href="/game/bos-vs-gsw-0042100405/box-score" id="box-score">Box Score</a>]

In [20]:
html = browser.html
test2 = soup(html, 'html.parser')
test2.find(class_="StatsTableHead_thead__2tFet")

<thead class="StatsTableHead_thead__2tFet"><tr><th>PLAYER</th><th class="text-right">MIN</th><th class="text-right">FGM</th><th class="text-right">FGA</th><th class="text-right">FG%</th><th class="text-right">3PM</th><th class="text-right">3PA</th><th class="text-right">3P%</th><th class="text-right">FTM</th><th class="text-right">FTA</th><th class="text-right">FT%</th><th class="text-right">OREB</th><th class="text-right">DREB</th><th class="text-right">REB</th><th class="text-right">AST</th><th class="text-right">STL</th><th class="text-right">BLK</th><th class="text-right">TO</th><th class="text-right">PF</th><th class="text-right">PTS</th><th class="text-right">+/-</th></tr></thead>

In [15]:
team_data

,MIN,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,...,REB,AST,STL,BLK,TO,PF,PTS,+/-,Date,Score
Harrison Barnes,13:03,1.0,2.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,2.0,11.0,04/10/2022,4.75
Trey Lyles,18:24,4.0,10.0,40.0,0.0,1.0,0.0,1.0,2.0,50.0,...,7.0,1.0,1.0,0.0,3.0,1.0,9.0,7.0,04/10/2022,19.75
Damian Jones,29:59,8.0,9.0,88.9,1.0,2.0,50.0,2.0,5.0,40.0,...,6.0,3.0,0.0,2.0,1.0,2.0,19.0,-1.0,04/10/2022,35.00
Justin Holiday,31:03,7.0,10.0,70.0,5.0,6.0,83.3,0.0,0.0,0.0,...,2.0,1.0,2.0,0.0,1.0,4.0,19.0,-5.0,04/10/2022,29.00
Davion Mitchell,39:34,5.0,14.0,35.7,2.0,4.0,50.0,6.0,8.0,75.0,...,3.0,15.0,0.0,2.0,1.0,1.0,18.0,-3.0,04/10/2022,48.75
Donte DiVincenzo,27:08,5.0,11.0,45.5,4.0,7.0,57.1,5.0,7.0,71.4,...,4.0,3.0,1.0,0.0,2.0,4.0,19.0,8.0,04/10/2022,31.50
Chimezie Metu,29:36,2.0,5.0,40.0,1.0,2.0,50.0,2.0,2.0,100.0,...,7.0,2.0,4.0,1.0,1.0,0.0,7.0,0.0,04/10/2022,28.75
Neemias Queta,18:01,3.0,4.0,75.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6.0,0.0,0.0,2.0,0.0,3.0,6.0,8.0,04/10/2022,17.50
Jeremy Lamb,15:29,2.0,3.0,66.7,1.0,1.0,100.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,3.0,1.0,5.0,2.0,04/10/2022,7.25
Josh Jackson,17:43,3.0,8.0,37.5,0.0,3.0,0.0,6.0,6.0,100.0,...,3.0,0.0,0.0,0.0,2.0,2.0,12.0,8.0,04/10/2022,14.75


In [4]:
#quit browser
browser.quit()